In [31]:
#設定讀取資料的年份與時段

y = 104     # '110'年或'104'年
t = '晨峰'    # '晨峰'或'昏峰'
case = '內湖比例吸引'    # '內湖轉移'或'北士科就近吸引'或'內湖比例吸引'

In [ ]:
#.xls to .csv file

import pandas as pd
import numpy as np
import os

if y == 110:
    input_file = 'TRTS4S_晨昏峰小時指派OD.xlsx'
if y == 104:
    input_file = 'TRTS4S_Y104晨昏峰小時指派OD_轉向_屏柵.xlsx'

output_file = f'{y}{t}_demand.csv'

if os.path.isfile(output_file):
    print(f"{output_file} 已存在，直接轉換為 dat 檔")
else:
    df = pd.read_excel(input_file, sheet_name=f'{y}年指派OD{t}小時')
    df.fillna(0, inplace=True)
    #save as csv
    df.to_csv(output_file, index=False, sep=',')
    print(f"Successfully saved {output_file}")

104晨峰_demand.csv 已存在，直接轉換為 dat 檔


## CRAZY DATA

In [22]:
# CRAZY DATA
toNeihu_carPCU = [52.57131377894737,
                  240.87165508421052,
                126.50885690526316,
                0.5117143473684211,
                44.60727214736842,
                14.263417810526317,
                7.830284231578948,
                48.00707641052632,
                99.59120320000001,
                1.2707160105263158,
                23.124324694736842,
                3.89035392631579,
                36.97841836842106,
                23.496752663157892,
                9.915908747368423,
                17.754764705263167,
                20.953982326315792,
                34.12894154736842,
                33.498218705263156,
                9.661538084210529,
                0.7814149578947369,
                0.41866819999999993,
                0.2118716315789474,
                0.13857555789473686,
                0.13114244210526316,
                0.2922874210526316,
                0.2414239157894737,
                1.0962516947368421,
                5.649780189473684,
                17.859298252631575,
                8.907277684210527,
                6.186926168421053,
                14.262367021052633,
                6.447190768421053,
                7.813624178947369,
                3.5862465473684217,
                7.238994115789474,
                6.107520621052631,
                10.896331536842109,
                2.64785614736842,
                8.13179932631579,
                0.011864736842105263,
                4.04895767368421,
                24.01648647368421,
                15.790579778947368,
                15.207920063157895,
                23.513276747368423,
                12.472439936842104,
                11.029774715789475,
                6.650019042105264,
                1.9071887157894736,
                37.3312866631579,
                9.510630673684211,
                7.012545442105264,
                2.6391933052631575,
                48.77157090526316,
                11.019298357894737,
                19.998922736842104,
                6.200701842105263,
                32.955435421052634,
                14.641730642105264,
                16.20070262105263,
                7.8873482,
                14.826689589473688,
                1.6570496421052634,
                3.5518740631578942,
                4.703724526315789,
                1.8647411999999999,
                5.294369094736842,
                3.4061110421052634,
                11.563238810526315,
                6.9723953157894725,
                3.0370765894736835,
                5.756013505263158,
                10.850252315789474,
                62.91968332631579,
                13.19074554736842,
                12.2119322,
                26.62407017894737,
                24.66132492631579,
                11.168475578947369,
                25.461273547368418,
                5.266450557894737,
                4.169205863157895,
                18.86304571578948,
                5.753614726315789,
                12.551531505263158,
                7.508880831578947,
                13.558876863157897,
                18.739096421052633,
                8.215166957894736,
                6.779142431578947,
                7.184274263157896,
                4.937814736842106,
                5.096805894736843,
                2.221346547368421,
                2.877610168421053,
                3.0408264000000003,
                1.8587089157894738,
                8.362119778947367,
                6.71962172631579,
                4.522535336842106,
                6.279300305263158,
                9.203948705263159,
                2.320025336842105,
                4.337495105263157,
                1.0026681052631579,
                3.3778032,
                7.296863663157895,
                0.492685052631579,
                0.4368682421052632,
                0.17352438947368418,
                3.1738472315789474,
                0.6993583368421055,
                0.7108240210526316,
                8.910655421052633,
                0.007453000000000001,
                5.007319484210526,
                1.6303880526315788,
                3.2242330842105265,
                5.712604336842106,
                3.5450996421052636,
                1.2006132210526317,
                8.003282842105264,
                2.952331494736842,
                11.50353272631579,
                15.029983231578953,
                12.279241442105263,
                7.390158263157895,
                4.995005663157895,
                29.89768653684211,
                75.30637355789476,
                34.97331706315789,
                58.46902149473685,
                94.03484230526315,
                22.90727053684211,
                19.606159905263155,
                5.647694989473684,
                77.5688723894737,
                53.4144797263158,
                109.43517867368422,
                141.26441231578949,
                56.23066402105264,
                65.99518461052632,
                20.02917672631579,
                19.312445894736843,
                48.95292682105263,
                88.83766694736842,
                61.849743399999994,
                36.8889324,
                70.98178914736843,
                65.89415302105263,
                114.87980736842106,
                14.013351157894737,
                7.0793215157894736,
                4.8789721578947365,
                9.897983736842104,
                7.410412052631578,
                5.082968421052631,
                29.484773052631578,
                11.187679968421053,
                30.74039237894737,
                9.79569314736842,
                6.569465105263158,
                42.81937264210526,
                23.35641783157895,
                2.3737843578947366,
                3.202177052631579,
                0.14445739999999999,
                0.002764463157894737,
                5.1322181473684205,
                1.0524940842105264,
                7.303520757894737,
                1.583988936842105,
                11.88840737894737,
                4.382909094736842,
                11.561480652631579,
                12.286664284210527,
                0.6347609789473683,
                2.358235557894737,
                2.0569954,
                5.17301514736842,
                2.8678302842105263,
                2.450622231578947,
                4.489235368421053,
                3.289475442105263,
                1.2256204421052628,
                2.9326907578947363,
                6.497200515789473,
                5.040820747368421,
                3.0612049684210527,
                2.282591557894737,
                2.604314010526316,
                1.1651673473684212,
                1.2931026210526315,
                0.7695299789473684,
                0.7391409894736841,
                0.33408051578947373,
                2.1720287263157894,
                0.5450292210526315,
                0.7500499894736842,
                4.330144105263158,
                6.385655800000002,
                5.240675031578948,
                2.007789505263158,
                3.5978904842105264,
                1.408254536842105,
                1.1187704947368422,
                0.5992311263157893,
                3.6308387052631583,
                1.607678947368421,
                1.1613939578947368,
                0.4459723789473684,
                6.509093042105263,
                4.683554894736842,
                0.5034246421052632,
                0.4619391157894737,
                0.5767875157894738,
                9.663968031578944,
                20.91566922105264,
                16.12376771578947,
                10.657047347368424,
                8.777645915789472,
                12.374696105263158,
                30.118328694736846,
                23.9926878,
                2.6335362315789475,
                10.210012968421054,
                17.88911168421053,
                0.5223761473684212,
                106.4293051894737,
                44.21016163157894,
                29.771985473684207,
                22.128720726315787,
                8.775087336842104,
                5.664269705263159,
                0.05018397894736843,
                26.221727,
                44.500830652631585,
                18.589733368421054,
                21.95871689473685,
                5.545885389473684,
                68.43726405263158,
                14.559989105263158,
                20.548463684210525,
                0.17582865263157893,
                4.223711778947369,
                6.838711042105262,
                0.034808863157894734,
                17.995502315789476,
                7.236488494736843,
                2.631017263157895,
                4.320741410526315,
                5.211690263157896,
                3.6783340736842103,
                30.48398362105263,
                12.099199073684211,
                2.909712726315789,
                0.23234225263157895,
                18.789438,
                3.767038252631579,
                21.432276052631575,
                24.889833757894746,
                19.539243157894735,
                48.058832589473674,
                17.56022792631579,
                18.984379989473688,
                16.96051347368421,
                0.026604136842105264,
                1.1104858947368426,
                14.852231978947371,
                1.6617523684210527,
                3.946031936842105,
                2.2816511999999998,
                1.0210542315789475,
                0.11563234736842107,
                0.010486557894736842,
                24.78875287368421,
                59.11730602105263,
                5.158198210526316,
                1.4758160315789475,
                0.8834632105263156,
                1.1662738210526316,
                4.939262610526316,
                4.398492084210526,
                2.543352336842105,
                0.5786575684210526,
                5.227792610526317,
                10.409806357894736,
                8.067258031578946,
                8.232998421052631,
                5.1787978842105264,
                4.32342494736842,
                4.3746933684210525,
                2.284472547368421,
                4.192100789473685,
                2.5218834421052625,
                11.532061336842103,
                5.215143494736842,
                4.99892772631579,
                2.520426684210526,
                5.09361605263158,
                3.8473134,
                0.8595452105263158,
                1.5887588631578948,
                0.9893773473684211,
                0.5123690631578948,
                0.3294050421052631,
                0.3731049157894737,
                0.0006421368421052633,
                0.7136732526315789,
                0.43668189473684205,
                0.2556788210526316,
                2.244333715789473,
                0.3812231684210526,
                2.1132102421052625,
                0.3244127578947368,
                0.27425855789473685,
                2.1401316210526318,
                2.0525952,
                8.656186631578949,
                7.701408084210525,
                6.339917368421052,
                3.9837940315789475,
                2.9200152631578944,
                5.596470736842106,
                9.951410242105265,
                2.6011268842105264,
                2.3265554631578946,
                2.776036094736842,
                4.189175642105264,
                7.198522852631578,
                8.516093536842105,
                1.8127897263157897,
                11.285050947368422,
                11.274341789473684,
                7.500418863157895,
                1.6365098315789472,
                1.116213294736842,
                3.738299368421052,
                0.15082376842105263,
                0.601694452631579,
                0.10270831578947369,
                0.28599206315789477,
                0.5287063157894737,
                0.0018928842105263158,
                0.6166062,
                0.5418274105263158,
                1.482013589473684,
                0.29768052631578945,
                0.35863402105263165,
                0.14687064210526315,
                0.7652874526315789,
                0.32024056842105264,
                0.4368171157894737,
                0.28572161052631573,
                0.40315610526315787,
                0.5526977473684211,
                0.9578259157894737,
                0.5070034315789474,
                0.34398721052631587,
                1.4572704210526315,
                1.0894050947368423,
                0.4515113368421052,
                0.12269015789473683,
                0.14103951578947369,
                0.8338976315789474,
                0.40661869473684215,
                1.2343048842105264,
                1.2848473789473684,
                1.7102895368421052,
                0.823818252631579,
                0.8186057578947369,
                0.7662981473684211,
                1.5302061473684208,
                0.24158251578947368,
                0.32431246315789475,
                0.5447586000000001,
                0.9029871894736842,
                1.0561452842105266,
                0.3084255368421052,
                0.15700941052631576,
                0.1900220421052631,
                0.20742250526315786,
                0.1819342842105263,
                0.157601,
                0.2903966421052632,
                0.6041038947368421,
                0.4893682842105263,
                0.44169036842105264,
                4.294563842105264,
                3.7953688736842106,
                4.926885978947368,
                0.2760793368421052,
                0.4736369789473684,
                0.6139863157894737,
                0.9947010842105263,
                2.961133610526316,
                0.7359913263157895,
                22.87735195789474,
                15.443652947368422,
                0.5703384842105261,
                3.8672033684210527,
                2.6419569789473685,
                2.846147105263158,
                1.8512739368421056,
                2.4498957473684215,
                2.8230113368421055,
                4.674629063157894,
                1.2286910421052633,
                1.0204817368421055,
                2.110870115789474,
                0.320194147368421,
                4.095526852631578,
                7.615489631578946,
                3.265832663157895,
                5.241092663157895,
                2.7452019578947366,
                10.93769722105263,
                3.686868568421052,
                3.5346209368421047,
                9.070766263157894,
                9.352145115789472,
                5.122910273684211,
                23.963061642105266,
                11.236491147368419,
                1.4647038736842104,
                34.263052252631574,
                2.8932893263157897,
                4.808687178947368,
                10.481634231578948,
                22.70462076842105,
                2.4999008842105264,
                1.046267494736842,
                11.279520084210526,
                3.1956863999999996,
                0.5056787263157895,
                1.538656052631579,
                4.599798621052631,
                30.084839694736843,
                2.0870145263157895,
                5.5203603999999995,
                4.059060968421052,
                9.15056122105263,
                13.274291073684207,
                15.515799168421054,
                22.986333863157896,
                24.119561526315785,
                2.985989852631579,
                1.5629773684210522,
                5.532018515789473,
                2.701709978947368,
                1.6047298947368418,
                3.823936168421053,
                1.0327850736842108,
                0.7221194842105263,
                0.3353666631578947,
                0.05078069473684211,
                0.09799966315789475,
                1.107199252631579,
                2.096716094736842,
                1.6644766421052632,
                1.1577138,
                2.0561766210526318,
                1.2299842210526313,
                1.1940055684210527,
                1.783238305263158,
                0.9586169578947369,
                5.244181357894734,
                11.780291242105262,
                21.23457872631579,
                10.501955042105262,
                7.284439452631578,
                7.920273652631578,
                3.0787895894736845,
                13.491980884210525,
                20.42690857894737,
                4.358511715789474,
                0.5071325578947369,
                2.973292263157894,
                5.411167105263158,
                0.5756864105263158,
                2.2169620315789476,
                0.30313526315789474,
                0.5453388842105262,
                0.3130762736842104,
                0.2007802315789474,
                0.13986439999999997,
                0.16625387368421052,
                0.17013525263157897,
                0.3446862210526315,
                0.34425172631578954,
                2.111313505263158,
                0.15061246315789475,
                2.4554640210526317,
                0.15503555789473686,
                0.10372747368421054,
                0.15171333684210525,
                0.8560914526315789,
                0.6583565684210527,
                0.9085594736842106,
                1.4903196105263157,
                5.247599705263158,
                5.529355094736841,
                1.0048511473684212,
                0.5876689684210527,
                0.33475184210526304,
                0.23588626315789477,
                12.115675431578946,
                4.542997357894737,
                37.76642737894737,
                25.660579726315795,
                78.52785418947369,
                126.88316269473684,
                36.202787094736834,
                41.54837057894738,
                51.385848515789476,
                2.531277168421053,
                0.40708565263157903,
                0.2353796421052632,
                40.51508923157895,
                29.121861410526314,
                1.6149910105263159,
                99.86656616842107,
                1.306784736842105,
                0.6447547052631579,
                14.258977294736843,
                24.617279884210525,
                1.7099361578947367,
                1.3952837578947368,
                0.558789147368421,
                1.3657159157894734,
                0.4632166105263158,
                0.21006478947368423,
                0.0867428947368421,
                2.460481978947368,
                1.0840538736842105,
                0.6817485789473684,
                25.405449178947368,
                2.995540663157895,
                3.4021005263157895,
                4.3223935052631575,
                2.148007010526316,
                4.948576115789472,
                0.4767913789473684,
                50.820695189473675,
                37.66622801052632,
                13.498254663157896,
                78.48856094736841,
                72.01264541052633,
                59.95730731578948,
                8.97026194736842,
                9.69472572631579,
                8.722629789473684,
                7.445554315789474,
                37.60570412631579,
                59.93863034736843,
                22.486261915789477,
                15.254434884210527,
                21.21907034736842,
                0.5411447368421052,
                0.09044881052631579,
                5.1e-05,
                2.337979905263158,
                10.08806650526316,
                0.9260033263157895,
                4.539808989473686,
                2.194230494736842,
                0.18542461052631581,
                0.34110115789473683,
                0.22911713684210525,
                0.3856317368421053,
                1.0727419999999999,
                1.2984013789473687,
                0.06383043157894738,
                0.9384807789473683,
                1.0412544421052632,
                0.18077485263157897,
                0.7763703368421053,
                0.5283696210526316,
                0.7259876526315789,
                4.579612589473684,
                1.486206842105263,
                5.396772336842105,
                22.395105852631573,
                21.57246522105263,
                3.2989363578947373,
                18.811647347368417,
                21.599192347368422,
                20.999724799999996,
                19.22375368421053,
                14.914221115789474,
                21.07587254736842,
                0.32511288421052625,
                0.727732663157895,
                0.42362458947368425,
                0.1153806,
                0.19286561052631576,
                0.2270578842105263,
                0.12035661052631577,
                0.4997652842105263,
                0.7787447894736842,
                0.32032471578947364,
                0.16531605263157892,
                0.2987428105263158,
                0.1556712210526316,
                0.16653498947368423,
                0.38778264210526314,
                2.920278810526316,
                0.21387791578947368,
                1.2931090421052631,
                2.4541644210526314,
                2.9892548526315794,
                0.04211025263157894,
                2.0597747789473684,
                4.082450978947368,
                2.5808200421052634,
                0.025282031578947373,
                0.09324838947368422,
                0.24988548421052634,
                1.2802829473684212,
                1.0284955999999998,
                0.8130162000000001,
                1.6681839263157896,
                2.4603597894736837,
                0.6143801157894737,
                2.0881079684210526,
                1.6790983263157893,
                4.257796094736842,
                4.105056231578947,
                1.2957322947368421,
                8.310117042105263,
                2.3867179684210527,
                0.432103452631579,
                0.2972837578947369,
                0.5705915052631579,
                0.18926846315789472,
                0.20397074736842108,
                0.024168484210526316,
                0.04857396842105263,
                0.6671717894736842,
                0.12912095789473685,
                0.275322,
                0.14878550526315787,
                0.2223243263157895,
                0.13773537894736843,
                0.07992518947368421,
                0.4508363157894737,
                0.00018999999999999993,
                0.20097809473684214,
                0.04991545263157895,
                0.26100126315789474,
                0.18647566315789477,
                0.35931553684210527,
                0.07660028421052631,
                0.21306592631578947,
                0.2697826947368421,
                0.10713530526315788,
                0.09837736842105263,
                0.10365409473684212,
                0.09743412631578947,
                0.13753220000000002,
                0.09621235789473685,
                0.08443884210526316,
                0.060243652631578944,
                0.23788183157894735,
                0.13198544210526317,
                0.3598294000000001,
                0.28874091578947364,
                0.4500298210526316,
                0.23308465263157896,
                0.43472307368421054,
                0.19731068421052633,
                0.2199630105263158,
                0.1179587052631579,
                0.3226639368421052,
                0.26091814736842106,
                0.31485346315789475,
                0.270516947368421,
                0.7919075578947368,
                0.0009609684210526316,
                0.0015521894736842106,
                0.0016958000000000003,
                0.002897768421052631,
                0.0022614842105263154,
                0.000869,
                0.28487937894736837,
                0.2747899368421053,
                0.04018798947368421,
                0.7150010947368421,
                0.24329488421052625,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.040189,
                12.383393999999996,
                95.297714,
                11.541343999999999,
                0.43553,
                11.965635,
                0.0026890000000000013,
                8.876956336842106,
                2.0957087052631582,
                4.284650589473684,
                0.319684,
                0.0013689999999999998,
                0.2218086210526316,
                36.097559000000004,
                1.1763969789473683,
                1.6668005052631583,
                8.159630705263158,
                0.5704271789473685,
                0.044694463157894736,
                0.05370702105263159,
                3.359656031578947,
                2.815785052631579,
                1.065736715789474,
                1.1020378105263158,
                12.403181505263161,
                3.9033418105263156,
                0.7939440000000001,
                1.075554,
                0.6611343368421052,
                1.4104178315789473,
                1.5184883157894737,
                2.2851663789473684,
                0.0,
                0.0]

toNeihu_motoPCU = [109.42418799999999,
 109.31908999999999,
 123.491118,
 0.0004999999999999999,
 63.712123999999996,
 21.608766000000006,
 14.657425,
 94.675128,
 140.80852299999998,
 5.209404000000001,
 42.787733,
 6.266392999999999,
 51.00529900000001,
 29.072935,
 12.466273,
 23.307397999999996,
 16.794027000000003,
 20.661229,
 19.024085999999997,
 5.042591,
 0.462861,
 0.16145,
 0.137033,
 0.10898100000000004,
 0.09826,
 0.15413300000000002,
 0.151023,
 0.6540339999999999,
 3.2392499999999997,
 7.781268000000001,
 4.730663,
 3.037311,
 6.285175000000001,
 3.706145,
 5.2157350000000005,
 1.786534,
 3.0282080000000002,
 4.1221440000000005,
 3.8183350000000003,
 0.7774140000000002,
 4.203367,
 0.001098,
 1.422536,
 10.337809,
 3.097147,
 4.127820999999999,
 35.733103,
 16.514223,
 12.999220000000001,
 7.121618000000001,
 0.178209,
 11.497759999999998,
 5.707337,
 7.173908000000001,
 0.23874499999999999,
 10.214815,
 2.8113900000000003,
 4.0230809999999995,
 4.151414,
 12.02571,
 19.663351,
 12.046940000000001,
 1.891103,
 7.538654,
 0.194474,
 0.408495,
 0.28469999999999995,
 0.252649,
 0.6047359999999998,
 0.384768,
 0.545038,
 0.466588,
 0.298787,
 0.9987989999999999,
 2.877675,
 10.884951,
 5.350048,
 5.510183,
 8.321384,
 11.110047,
 4.893046999999999,
 33.941122,
 10.183390000000001,
 1.610977,
 21.00529,
 4.141243,
 30.654306,
 10.323803,
 3.011812,
 4.401514000000001,
 1.779897,
 2.285323,
 4.919346,
 1.9515039999999997,
 18.752201,
 6.06903,
 7.482351999999999,
 11.859576,
 1.0833110000000001,
 1.1995109999999998,
 2.440691,
 0.7787450000000001,
 4.445772,
 5.559245,
 1.8061589999999998,
 1.9427089999999998,
 0.30417,
 0.5392560000000001,
 0.6423180000000001,
 0.19398899999999997,
 0.178645,
 0.133748,
 0.9688140000000001,
 0.26918900000000007,
 0.167649,
 0.20763600000000001,
 0.0,
 0.507995,
 0.31358600000000003,
 1.0889399999999998,
 1.1713719999999999,
 1.10014,
 0.376713,
 1.352137,
 0.6339800000000001,
 3.3100899999999993,
 1.1844890000000001,
 1.4814610000000004,
 0.678923,
 2.23677,
 73.860344,
 147.578183,
 73.70128100000001,
 110.453648,
 180.755473,
 42.458574999999996,
 39.357763999999996,
 9.820874999999997,
 135.83351700000003,
 113.15253,
 140.075504,
 157.91145900000004,
 47.758643000000006,
 49.493561,
 10.363335,
 10.978459999999998,
 26.581889999999994,
 58.695323,
 45.31069500000001,
 29.785670000000003,
 69.06568800000001,
 89.48599700000001,
 219.07263999999998,
 12.146901999999999,
 7.1825980000000005,
 5.970044,
 7.271072,
 7.682807,
 4.4598189999999995,
 46.956794,
 18.526503,
 50.97404,
 11.643947999999998,
 13.219367,
 57.571432,
 29.272951000000003,
 3.4288050000000005,
 1.776447,
 0.037125000000000005,
 0.001332,
 2.400921,
 3.255664,
 3.6497790000000006,
 3.959879000000001,
 5.924803999999999,
 2.461253,
 9.072578,
 11.839426000000001,
 0.11089499999999998,
 5.329175,
 7.668856000000001,
 6.294456,
 6.807510000000001,
 6.005717,
 6.723694999999999,
 3.764031,
 2.4291859999999996,
 4.576578,
 6.387704000000001,
 5.484549999999999,
 3.951154,
 2.4319949999999992,
 3.165183999999999,
 1.2565529999999998,
 1.1948330000000005,
 0.616716,
 0.618215,
 0.388961,
 2.716048,
 0.5336759999999999,
 0.767459,
 4.0421629999999995,
 5.848138,
 5.1269789999999995,
 2.1036680000000003,
 4.079605999999999,
 1.9441739999999998,
 1.8272110000000004,
 0.828515,
 5.029680999999999,
 0.916462,
 0.497404,
 0.28701000000000004,
 18.433053,
 13.427096,
 0.31110600000000005,
 0.019417,
 0.37636900000000006,
 38.372796,
 61.420879,
 50.692902000000004,
 16.090716,
 19.750111,
 50.370599,
 39.29466,
 32.816846,
 9.409626,
 14.734651000000001,
 9.605557,
 0.006466,
 15.770696000000001,
 31.743281000000003,
 33.215001,
 21.363730999999998,
 11.089811,
 2.915378,
 0.009507000000000002,
 8.124292,
 15.090897,
 9.649715,
 8.947149,
 4.055642,
 27.953775999999998,
 10.994444,
 6.655339,
 0.001026,
 1.338297,
 1.2749969999999997,
 0.0004540000000000001,
 6.717007,
 6.020253,
 2.815259,
 1.7573679999999998,
 3.28806,
 2.238448,
 5.5874500000000005,
 5.543671000000001,
 0.6722349999999999,
 0.019911,
 7.95532,
 3.1579099999999998,
 14.635107999999997,
 6.695856,
 5.324222000000001,
 11.190275999999999,
 7.619968,
 5.186653,
 21.031094,
 0.008086000000000001,
 1.1008489999999997,
 11.829721000000003,
 2.3975820000000003,
 6.330240000000001,
 2.05056,
 0.9209320000000001,
 0.23769,
 9.8e-05,
 11.3157,
 25.211123,
 7.544409,
 3.392602,
 2.3245829999999996,
 2.919221,
 6.164194,
 9.626194,
 3.0168060000000003,
 1.438024,
 8.836181999999999,
 1.0636610000000002,
 0.9638209999999999,
 1.1260320000000001,
 0.611851,
 0.514517,
 0.593696,
 0.44240000000000007,
 0.655835,
 0.505123,
 1.766074,
 0.8099939999999998,
 0.7438850000000001,
 0.44736000000000004,
 0.8055180000000001,
 0.50102,
 0.043843999999999994,
 0.102842,
 0.046303000000000004,
 0.024077,
 0.033440000000000004,
 0.021469000000000002,
 0.0023279999999999993,
 0.018128000000000005,
 0.01462,
 0.008327,
 0.05101399999999999,
 0.008547,
 5.354933,
 0.862478,
 0.715868,
 0.26819800000000005,
 0.25981000000000004,
 2.158128,
 1.9452319999999996,
 1.102706,
 0.808619,
 0.7479109999999999,
 1.350542,
 1.7938070000000002,
 0.958193,
 0.6041049999999999,
 0.778943,
 1.4593809999999998,
 1.8892689999999999,
 2.27951,
 0.39327500000000004,
 2.9825189999999995,
 3.308389,
 2.951415,
 0.21900899999999998,
 0.575234,
 0.718143,
 0.19089399999999998,
 0.39966899999999994,
 0.108038,
 0.313571,
 0.32006599999999996,
 0.0025359999999999996,
 1.0199340000000001,
 0.366846,
 0.602612,
 0.145178,
 0.43072499999999997,
 0.145992,
 1.049323,
 0.5361,
 0.8655959999999999,
 0.470608,
 0.6054710000000001,
 0.756741,
 1.108662,
 0.6259410000000001,
 0.24479,
 0.544919,
 0.283129,
 0.13052800000000003,
 0.017966,
 0.041083999999999996,
 0.12957500000000002,
 0.13785300000000003,
 0.692896,
 0.73049,
 0.8063190000000001,
 0.6938909999999999,
 0.5306839999999999,
 0.562392,
 1.304848,
 0.275287,
 0.35486600000000007,
 0.44282,
 0.496442,
 0.7159680000000002,
 0.216893,
 0.08817399999999999,
 0.128547,
 0.09341500000000001,
 0.229227,
 0.254006,
 0.515579,
 0.6816880000000002,
 0.785445,
 0.360781,
 0.075039,
 0.06960000000000001,
 0.071026,
 0.009084,
 0.004338000000000001,
 0.07023199999999999,
 0.028878999999999995,
 0.033165,
 0.004832,
 0.40394299999999994,
 0.14373999999999998,
 5.015900000000001,
 2.0482609999999997,
 1.212232,
 1.709379,
 0.9561360000000001,
 1.0084710000000001,
 1.632456,
 2.9190530000000003,
 0.537969,
 0.4051550000000001,
 1.116982,
 3.9252879999999992,
 1.104773,
 34.25684699999999,
 9.378267000000001,
 5.914408000000002,
 2.953339,
 10.666237,
 8.242951,
 6.8669590000000005,
 15.625594999999999,
 31.778394000000002,
 10.061582000000001,
 95.430113,
 24.108136999999996,
 3.067364,
 39.187224,
 0.470327,
 0.09255300000000001,
 0.124193,
 0.4331689999999999,
 0.013647,
 0.006538,
 0.025007,
 0.012499,
 0.024068,
 0.018149999999999996,
 0.16764700000000002,
 0.166974,
 0.023575,
 0.04967800000000001,
 0.13854599999999997,
 1.332792,
 2.5677000000000003,
 1.3614940000000004,
 1.531042,
 1.3990169999999997,
 0.8261170000000001,
 1.0238200000000002,
 1.009124,
 1.097361,
 0.6400049999999999,
 0.9860069999999999,
 0.795924,
 0.47108200000000006,
 0.37818199999999996,
 0.000226,
 4.800000000000001e-05,
 1.8051739999999998,
 1.368358,
 1.2339929999999997,
 1.266132,
 2.7579930000000004,
 0.891634,
 0.506402,
 0.619621,
 0.15499000000000002,
 58.697247000000004,
 0.041335000000000004,
 0.066029,
 0.045394,
 0.05105799999999999,
 0.035877,
 0.019695,
 0.069151,
 0.07283499999999998,
 0.022906,
 0.020589,
 0.697348,
 0.45843199999999995,
 0.8982899999999998,
 4.723907,
 0.492026,
 1.6462719999999997,
 0.8717239999999999,
 0.5811780000000001,
 0.405316,
 0.400379,
 0.34656999999999993,
 1.2502609999999998,
 1.011275,
 4.609851,
 0.413163,
 9.895109,
 0.33905899999999994,
 0.242417,
 0.3578680000000001,
 2.08034,
 0.947982,
 1.434662,
 2.320379,
 10.132531999999998,
 16.343609999999998,
 3.818348,
 1.911367,
 1.5517889999999999,
 0.645084,
 4.920312,
 2.679748,
 4.282274,
 11.790185999999999,
 7.051836,
 7.012536,
 2.7920520000000004,
 7.9488970000000005,
 3.3588080000000002,
 0.8098320000000001,
 0.184234,
 0.143759,
 6.505855,
 2.803748,
 0.534671,
 6.716674999999999,
 1.026413,
 0.35434199999999993,
 1.02202,
 1.575779,
 0.608048,
 0.7624350000000001,
 0.2369,
 0.34756400000000004,
 0.188364,
 0.092241,
 0.037853,
 0.7647120000000001,
 0.512487,
 0.261293,
 1.31492,
 0.20195800000000003,
 0.139934,
 0.166677,
 0.07619800000000002,
 0.617362,
 0.15201399999999998,
 1.0339790000000002,
 1.7583119999999999,
 1.034037,
 1.4976500000000001,
 1.8895160000000002,
 1.268954,
 0.6718679999999999,
 0.33603200000000005,
 0.129529,
 0.402741,
 0.5603920000000001,
 1.5470520000000003,
 0.32692999999999994,
 0.31160000000000004,
 0.28509999999999996,
 0.022750999999999997,
 0.033957,
 1.1000000000000001e-05,
 0.344564,
 0.40259500000000004,
 0.615639,
 29.383407,
 0.48608199999999996,
 0.094564,
 0.22236199999999998,
 0.136861,
 0.28953799999999996,
 0.572854,
 0.5584210000000001,
 0.051647,
 0.42150099999999996,
 0.38890600000000003,
 0.09962299999999999,
 0.5027630000000001,
 0.21690999999999996,
 0.548851,
 0.7218309999999999,
 0.22917600000000005,
 0.603484,
 1.1304610000000002,
 1.4365080000000001,
 0.11794,
 1.1589110000000002,
 0.9652029999999999,
 0.7272400000000001,
 0.341227,
 0.609846,
 0.410517,
 0.302052,
 0.45021599999999995,
 0.27009299999999997,
 0.105926,
 0.168745,
 0.281216,
 0.105523,
 0.264886,
 0.432331,
 0.16376399999999997,
 0.09809700000000002,
 0.15326800000000002,
 0.09875400000000001,
 0.159565,
 0.452209,
 0.00933,
 0.00125,
 0.059649,
 0.06537100000000001,
 0.025613,
 0.000294,
 0.029730000000000003,
 0.038571999999999995,
 0.022136,
 0.0017699999999999999,
 0.017154,
 0.003978,
 0.006125,
 0.006585,
 0.014235000000000001,
 0.048875999999999996,
 0.0025930000000000007,
 0.020828999999999997,
 0.014286,
 0.011447,
 0.02976,
 0.01573,
 0.06195300000000001,
 0.013605999999999998,
 0.01712,
 0.010092999999999998,
 0.002078,
 0.002754,
 0.000944,
 0.002334,
 0.000217,
 0.00028800000000000006,
 0.0062050000000000004,
 0.000773,
 0.0009180000000000001,
 0.000466,
 0.0006840000000000001,
 0.000468,
 0.000501,
 0.003985000000000001,
 0.0,
 0.000999,
 0.00021899999999999998,
 0.002219,
 0.0008870000000000001,
 0.0008550000000000001,
 0.00035099999999999997,
 0.0008730000000000002,
 0.000801,
 0.00031700000000000006,
 0.000411,
 0.000492,
 0.000332,
 0.0009209999999999999,
 0.000362,
 0.000298,
 0.000226,
 0.0012999999999999997,
 0.000821,
 0.001683,
 0.001241,
 0.0015809999999999997,
 0.0016500000000000002,
 0.004225,
 0.0013269999999999998,
 0.0014070000000000003,
 0.0007560000000000002,
 0.001994,
 0.001935,
 0.0029990000000000004,
 0.0028910000000000003,
 0.003229,
 5.2000000000000004e-05,
 2.5000000000000005e-05,
 1.4000000000000003e-05,
 9.6e-05,
 1.5000000000000002e-05,
 0.0,
 0.001465,
 0.0012909999999999998,
 0.00012800000000000002,
 0.0051730000000000005,
 0.0021750000000000003,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 28.189086,
 0.0,
 0.001676,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.032377,
 0.0,
 0.0,
 0.0,
 0.0]

## CODE

In [32]:
#.csv to .dat file

from tqdm import tqdm
import pandas as pd
import numpy as np

df = pd.read_csv(f'{y}{t}_demand.csv')
df.fillna(0, inplace=True)
rows = []

#482-502
tamsui = [i for i in range(482,503)]

count = 0

# Modification for Beitou-Shihlin Technical Park (BSTP)
Nsum = 0
BSTP = [207,209,213]  
BSTPNeighbor = [207,209,213] 
Neihu = [146,148,149,150,154,155,156,157,158,159]

# 運具分配 [晨峰, 昏峰]
car_percent = [0.292, 0.3117]
moto_percent = [0.359, 0.3907]

# 乘載率 [晨峰, 昏峰]
car_load_rate = [1.698, 1.678]
moto_load_rate = [1.139, 1.209]

# 當量
car_equiv = 1
moto_equiv = 0.3

# CASE: 內湖轉移
Neihu_tranfer = 35000
Neihu_dest_sum = 7840
avg_deduction = Neihu_tranfer / Neihu_dest_sum
BSTP_transfer = 35000
BSTP_dest_sum = 2352
BSTP_increase = BSTP_transfer / BSTP_dest_sum

# CASE: 北士科就近吸引
BSTPNeighbor_increase = BSTP_transfer / len(BSTPNeighbor)

# CASE: 內湖比例吸引
Neihu_carPCU_sum = 7087.399478126333
Neihu_motoPCU_sum = 5263.020866000018

# Helper function for converting PCU
if t == '晨峰':
    tp = 0
elif t == '昏峰':
    tp = 1

# PCU Conversion (time_period: 0 for '晨峰', 1 for '昏峰')
def convert_to_pcu(demand, veh_type, time_period):
    if veh_type == '1':  # Car
        return demand * car_percent[time_period] / car_load_rate[time_period] * car_equiv
    elif veh_type == '2':  # Motorcycle
        return demand * moto_percent[time_period] / moto_load_rate[time_period] * moto_equiv
    else:
        return 0

#use tqdm to show the progress for 614656 rows
print("demand.dat is generating...")
for index, row in tqdm(df.iterrows(), total=len(df)):
    origin = row["I"]
    dest = row["J"]
    moto = row["MOTO"]
    car = row["ALL"] - moto
    bus = row["BUS"]
    if t == '晨峰':
        car_demand = car + bus*1.8/19.0
    else:
        car_demand = car + bus*1.8/20.7

    out_of_cordon=[i for i in range(692,751)]+[783,784]
    if origin not in out_of_cordon and dest not in out_of_cordon:
        
        if case == '內湖轉移':
            if dest in Neihu:
                car_demand = car_demand - convert_to_pcu(avg_deduction, '1', tp)
                moto = moto - convert_to_pcu(avg_deduction, '2', tp)
            
            if dest in BSTP:
                car_demand = car_demand + convert_to_pcu(BSTP_increase, '1', tp)
                moto = moto + convert_to_pcu(BSTP_increase, '2', tp)

        elif case == '北士科就近吸引':
            if dest in BSTP and origin in BSTPNeighbor:
                car_demand = car_demand + convert_to_pcu(BSTPNeighbor_increase, '1', tp)
                moto = moto + convert_to_pcu(BSTPNeighbor_increase, '2', tp)

        elif case == '內湖比例吸引':
            if dest in BSTP:
                #print(f"origin: {origin}, dest: {dest}, car_demand: {car_demand}")
                car_distribution = BSTP_transfer / len(BSTP) * toNeihu_carPCU[int(origin-1)] / sum(toNeihu_carPCU)
                #print(f"car_distribution: {car_distribution}, origin: {origin}")
                car_demand = car_demand + convert_to_pcu(car_distribution, '1', tp)
                #print(f"car_demand after distribution: {car_demand}")

                #print(f"origin: {origin}, dest: {dest}, moto: {moto}")
                moto_distribution = BSTP_transfer / len(BSTP) * toNeihu_motoPCU[int(origin-1)] / sum(toNeihu_motoPCU)
                #print(f"moto_distribution: {moto_distribution}, origin: {origin}")
                moto = moto + convert_to_pcu(moto_distribution, '2', tp)
                #print(f"moto after distribution: {moto}")

        rows.append({'origin':int(origin),'dest':int(dest),'demand':car_demand,'vehType': '1'})
        rows.append({'origin':int(origin),'dest':int(dest),'demand':moto,'vehType': '2'})

print("demand.dat is generated")
df_out = pd.DataFrame(rows)
df_out.to_csv(f'{y}{t}{case}_demand.dat', index=False, sep='\t')

demand.dat is generating...


100%|██████████| 614656/614656 [00:10<00:00, 58903.22it/s]


demand.dat is generated


In [30]:
toNeihu_carPCU

[52.57131377894737,
 240.87165508421052,
 126.50885690526316,
 0.5117143473684211,
 44.60727214736842,
 14.263417810526317,
 7.830284231578948,
 48.00707641052632,
 99.59120320000001,
 1.2707160105263158,
 23.124324694736842,
 3.89035392631579,
 36.97841836842106,
 23.496752663157892,
 9.915908747368423,
 17.754764705263167,
 20.953982326315792,
 34.12894154736842,
 33.498218705263156,
 9.661538084210529,
 0.7814149578947369,
 0.41866819999999993,
 0.2118716315789474,
 0.13857555789473686,
 0.13114244210526316,
 0.2922874210526316,
 0.2414239157894737,
 1.0962516947368421,
 5.649780189473684,
 17.859298252631575,
 8.907277684210527,
 6.186926168421053,
 14.262367021052633,
 6.447190768421053,
 7.813624178947369,
 3.5862465473684217,
 7.238994115789474,
 6.107520621052631,
 10.896331536842109,
 2.64785614736842,
 8.13179932631579,
 0.011864736842105263,
 4.04895767368421,
 24.01648647368421,
 15.790579778947368,
 15.207920063157895,
 23.513276747368423,
 12.472439936842104,
 11.02977471

In [ ]:
# DEPRECATED
    # if origin within 691 and dest within 691, then demand *= 1.5
    # if origin <691 and dest <691:
        # demand *= 1.25
    # if origin in 304-691 or dest in 304-691, then demand *= 1.5
    # if origin <= 691 and dest <= 691 and origin >= 304 and dest >= 304:
        # demand *= 1.4
    # if dest within 303, then demand *= 1.5
    # if dest <= 303:
    #     demand *= 1
    # # if origin in tamsui or dest in tamsui:
    # if origin in tamsui:
    #     demand *= 1